# sklearn-porter

Repository: [https://github.com/nok/sklearn-porter](https://github.com/nok/sklearn-porter)

## LinearSVC

Documentation: [sklearn.svm.LinearSVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html)

In [1]:
import sys
sys.path.append('../../../../..')

### Load data

In [2]:
from sklearn.datasets import load_iris

iris_data = load_iris()

X = iris_data.data
y = iris_data.target

print(X.shape, y.shape)

((150, 4), (150,))


### Train classifier

In [3]:
from sklearn import svm

clf = svm.LinearSVC(C=1., random_state=0)
clf.fit(X, y)

/opt/miniconda/envs/sklearn-porter/lib/python2.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)

### Transpile classifier

In [4]:
from sklearn_porter import Porter

porter = Porter(clf, language='c')
output = porter.export()

print(output)

#include <stdlib.h>
#include <stdio.h>
#include <math.h>

double coefficients[3][4] = {{0.1842387105153816, 0.45123226735416666, -0.8079442418237148, -0.4507120994138801}, {0.05538075463135362, -0.9011091317807134, 0.40964725305179495, -0.9616708418438406}, {-0.8507797840387784, -0.9866797559823655, 1.380978948733964, 1.865367212912733}};
double intercepts[3] = {0.10956037385996253, 1.6860176665898299, -1.7096500573837932};

int predict (float features[]) {
    double class_val = -INFINITY;
    int class_idx = -1;
    int i, il, j, jl;
    for (i = 0, il = sizeof(coefficients) / sizeof (coefficients[0]); i < il; i++) {
        double prob = 0.;
        for (j = 0, jl = sizeof(coefficients[0]) / sizeof (coefficients[0][0]); j < jl; j++) {
            prob += coefficients[i][j] * features[j];
        }
        if (prob + intercepts[i] > class_val) {
            class_val = prob + intercepts[i];
            class_idx = i;
        }
    }
    return class_idx;
}

int main(int argc, const c

### Run classification in C

In [5]:
# Save model:
# with open('linearsvc.c', 'w') as f:
#     f.write(output)

# Compile model:
# $ gcc linearsvc.c -std=c99 -lm -o linearsvc

# Run classification:
# $ ./linearsvc 1 2 3 4